# Data preperation DBLP to OpenAlex

In [1]:
import json
import random
import requests
import time
import re
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict


### DBLP Query selection 
**Purpose**
This script extracts a balanced selection of 100 questions from the DBLP-QuAD dataset, ensuring that:

- Only specific templates (TP01, TP02, etc.) are included, all questions which can also be answered with OpenAlex.
- Questions from TP17 have a valid ORCID (fetched via the DBLP SPARQL endpoint). For those it makes sense to include the ORCID, for later creating promts including the ORCID for OpenAlex
- Questions are evenly distributed among template IDs.

**Purpose**
The goal of this script is to execute SPARQL queries from DBLP_100_questions.json against the DBLP SPARQL endpoint and store the responses. This allows for:
- Retrieving structured knowledge (e.g., authors, publications, venues) from DBLP.
- Ensuring data completeness by associating each question with its corresponding SPARQL result.
- Generating a structured dataset (DBLP_100_results.json) that combines questions and responses for further analysis.

Next Steps: 
- Query the queries in OpenAlex and get the corresponding results.

In [ ]:
# URL of the JSON dataset
URL = "https://raw.githubusercontent.com/awalesushil/DBLP-QuAD/refs/heads/main/data/DBLP-QuAD/train/questions.json"

# Define the SPARQL endpoint
SPARQL_ENDPOINT = "https://sparql.dblp.org/sparql"

# Fetch the JSON data from the URL
response = requests.get(URL)
data = response.json()

# List of template_ids to filter
TEMPLATE_IDS = {
    "TC01", "TC11", "TC12", "TC14", "TC15", "TC21", "TC24", "TC25",
    "TC53", "TC61", "TC71", "TC73", "TC75", "TC76"
}

SPECIAL_CASE_TEMPLATES = {"TC12", "TC53", "TC61"}  # Templates requiring two ORCIDs

# Filter questions by template_id
filtered_questions = defaultdict(list)
for q in data["questions"]:
    if q["template_id"] in TEMPLATE_IDS:
        filtered_questions[q["template_id"]].append(q)

# Cache for ORCID lookups to avoid redundant API calls
orcid_cache = {}

# Function to query SPARQL endpoint for ORCID using full author URL
def get_orcid_from_sparql(author_url, max_retries=3):
    """Fetch ORCID from SPARQL and retry if needed."""
    if author_url in orcid_cache:
        return orcid_cache[author_url]  # Use cached value

    query = f"""
    SELECT ?orcid WHERE {{
      <{author_url}> <https://dblp.org/rdf/schema#orcid> ?orcid
    }}
    """
    params = {"query": query, "format": "json"}

    for attempt in range(max_retries):
        try:
            response = requests.get(SPARQL_ENDPOINT, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
            bindings = data.get("results", {}).get("bindings", [])

            if bindings:
                orcids = [b["orcid"]["value"] for b in bindings]
                orcid_cache[author_url] = orcids  # Cache ORCID list
                return orcids

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt+1}: Error querying ORCID for {author_url}: {e}")
            time.sleep(1)  # Short delay before retrying

    orcid_cache[author_url] = []  # Store empty list if no ORCID found
    return []

# Process questions for each template_id
for template_id, questions in filtered_questions.items():
    valid_questions = []
    
    for question in questions:
        updated_entities = []
        author_entities = []

        for entity in question["entities"]:
            entity_clean = entity.strip("<>")
            if "/pid/" in entity_clean:  # Identify author entities
                orcids = get_orcid_from_sparql(entity_clean)
                if orcids:
                    author_entities.append({"author": entity, "orcid": orcids})
                else:
                    print(f"Warning: No ORCID found for {entity_clean}, skipping entity.")

                time.sleep(0.5)
            else:
                updated_entities.append({"entity": entity})

        # Ensure the required number of ORCIDs
        if template_id in SPECIAL_CASE_TEMPLATES:
            if len(author_entities) == 2 and all("orcid" in e and e["orcid"] for e in author_entities):
                updated_entities.extend(author_entities)
                question["entities"] = updated_entities
                valid_questions.append(question)
            else:
                print(f"Skipping question (template {template_id}) due to missing ORCID for both authors: {question['entities']}")
        else:
            if author_entities:
                updated_entities.extend(author_entities)
                question["entities"] = updated_entities
                valid_questions.append(question)
    
    filtered_questions[template_id] = valid_questions

# Determine the number of entries to select from each template_id
num_entries_per_template = 100 // len(TEMPLATE_IDS)

# Randomly select questions for each template_id
selected_questions = {}
for tid, questions in filtered_questions.items():
    selected_questions[tid] = random.sample(questions, min(num_entries_per_template, len(questions))) if questions else []

# Flatten the list of selected questions
final_selection = [q for questions in selected_questions.values() for q in questions]

# If the total number of selected questions is less than 100, sample additional questions
remaining_questions = [
    q for q in data["questions"] if q["template_id"] in TEMPLATE_IDS and q not in final_selection
]

additional_needed = 100 - len(final_selection)
if additional_needed > 0 and remaining_questions:
    final_selection.extend(random.sample(remaining_questions, min(additional_needed, len(remaining_questions))))

final_selection = final_selection[:100]  # Ensure exactly 100 questions

# Count the number of questions per template_id
template_counts = {tid: sum(1 for q in final_selection if q["template_id"] == tid) for tid in TEMPLATE_IDS}

# Print the counts per template_id
print("\nNumber of selected questions per template_id:")
for tid, count in sorted(template_counts.items()):
    print(f"{tid}: {count}")

# Create the final JSON structure
final_data = {"questions": final_selection}

# Save to a JSON file
output_filename = "DBLP_100_questions.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(final_data, f, indent=4, ensure_ascii=False)

print(f"\nTotal selected questions: {len(final_selection)}")
print(f"Saved selected questions to {output_filename}")

Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/83/11395>', '<https://dblp.org/pid/83/10230>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/27/7440>', '<https://dblp.org/pid/219/1025>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/03/4088>', '<https://dblp.org/pid/22/101>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/43/2890>', '<https://dblp.org/pid/10/8200>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/60/2019>', '<https://dblp.org/pid/21/3471>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/91/4616>', '<https://dblp.org/pid/41/4810>']
Skipping question (template TC12) due to missing ORCID for both authors: ['<https://dblp.org/pid/140/7900>', '<https://dblp.org/pid/140/7914>']
Sk

In [ ]:
SPARQL_ENDPOINT = "https://sparql.dblp.org/sparql"

# Load queries from the JSON file
input_filename = "DBLP_100_questions.json"
output_filename = "DBLP_100_results.json"
error_filename = "DBLP_100_errors.json"
error_count_filename = "DBLP_100_error_counts.json"  # New file for failure counts

with open(input_filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Store results
results = []
errors = []  # Store erroneous queries separately
error_counts = defaultdict(int)  # Count failures per template

# Iterate over each question and execute the SPARQL query
for question in data["questions"]:
    query = question["query"]["sparql"]  # Extract the SPARQL query
    query_id = question["id"]  # Get the question ID
    template_id = question.get("template_id", "unknown")  # Extract template ID (default to "unknown" if missing)

    # Define the request parameters
    params = {
        "query": query,
        "format": "json"
    }

    print(f"Querying {query_id} (Template {template_id})...")

    try:
        # Send request to the DBLP SPARQL endpoint
        response = requests.get(SPARQL_ENDPOINT, params=params, timeout=10)
        response.raise_for_status()
        query_result = response.json()

        # Store the original question along with the response
        question_with_response = question.copy()  # Preserve all original fields
        question_with_response["response"] = query_result  # Add API response

        results.append(question_with_response)

    except requests.exceptions.RequestException as e:
        print(f"Error querying {query_id} (Template {template_id}): {e}")

        # Store the erroneous query separately
        question_with_error = question.copy()
        question_with_error["response"] = {"error": str(e)}

        errors.append(question_with_error)  # Append to errors list

        # Count errors per template ID
        error_counts[template_id] += 1

# Save the results to a JSON file
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump({"questions": results}, f, indent=4, ensure_ascii=False)

# Save the errors to a separate JSON file
with open(error_filename, "w", encoding="utf-8") as f:
    json.dump({"questions": errors}, f, indent=4, ensure_ascii=False)

# Save error counts per template ID
with open(error_count_filename, "w", encoding="utf-8") as f:
    json.dump(error_counts, f, indent=4, ensure_ascii=False)

# Print summary of failed queries per template
print(f"\nSaved query results to {output_filename}")
print(f"Saved erroneous queries to {error_filename}")
print(f"Saved error counts to {error_count_filename}")

print("\nSummary of failed queries per template:")
for template, count in sorted(error_counts.items(), key=lambda x: -x[1]):
    print(f"Template {template}: {count} failures")

Querying Q0172 (Template TC01)...
Querying Q0101 (Template TC01)...
Querying Q0241 (Template TC01)...
Querying Q0123 (Template TC01)...
Querying Q0317 (Template TC01)...
Querying Q0291 (Template TC01)...
Querying Q0136 (Template TC01)...
Querying Q0422 (Template TC15)...
Querying Q0391 (Template TC15)...
Querying Q0539 (Template TC15)...
Querying Q0424 (Template TC15)...
Querying Q0410 (Template TC15)...
Querying Q0365 (Template TC15)...
Querying Q0503 (Template TC15)...
Querying Q0617 (Template TC11)...
Querying Q0632 (Template TC11)...
Querying Q0629 (Template TC11)...
Querying Q0684 (Template TC11)...
Querying Q0440 (Template TC11)...
Querying Q0606 (Template TC11)...
Querying Q0549 (Template TC11)...
Querying Q0524 (Template TC12)...
Querying Q0502 (Template TC12)...
Querying Q0360 (Template TC12)...
Querying Q0470 (Template TC12)...
Querying Q0594 (Template TC12)...
Querying Q0384 (Template TC12)...
Querying Q0408 (Template TC12)...
Querying Q0370 (Template TC14)...
Querying Q0486

Get the templates for the used questions to translate them to OpenAlex queries.

In [ ]:
# GitHub raw file URL
github_url = "https://raw.githubusercontent.com/awalesushil/DBLP-QuAD/main/templates.py"

# Template IDs to extract
template_ids = {
    "TC01", "TC11", "TC12", "TC14", "TC15", "TC21", "TC24", "TC25", "TC53", "TC61", "TC71", "TC73", "TC75", "TC76"
}

# Fetch the content from GitHub
response = requests.get(github_url)
response.raise_for_status()

# Extract the Python dictionary from the file content
file_content = response.text

# Execute the file content safely to obtain the `templates` dictionary
namespace = {}
exec(file_content, namespace)

# Extract the templates dictionary
templates_dict = namespace.get("templates", {})

# Extract the relevant templates
filtered_templates = {}
for category, entries in templates_dict.items():
    for key in ["SINGLE_FACT", "MULTI_FACT", "DOUBLE_INTENT", "DOUBLE_NEGATION", "UNION", "COUNT"]:
        if key in entries:
            for entry in entries[key]:
                if entry["id"] in template_ids:
                    filtered_templates[entry["id"]] = entry

# Save to JSON file
output_filename = "filtered_templates.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(filtered_templates, f, indent=4, ensure_ascii=False)

print(f"Filtered templates saved to {output_filename}")

Filtered templates saved to filtered_templates.json


Map the DBLP Questions to OpenAlex templates.

In [78]:
# File paths
dblp_file = "DBLP_100_questions.json"
openalex_template_file = "manual_query_translation_DBLP_OpenAlex.json"
output_file = "translated_queries_DBLP_OpenAlex.json"

# Load DBLP questions
with open(dblp_file, "r", encoding="utf-8") as f:
    dblp_data = json.load(f)["questions"]

# Load OpenAlex templates
with open(openalex_template_file, "r", encoding="utf-8") as f:
    openalex_templates = {t["template_id"]: t["sparql_OpenAlex"] for t in json.load(f)}

# Process each DBLP question and generate OpenAlex queries
translated_queries = []
for question in dblp_data:
    template_id = question["template_id"]
    entities = question.get("entities", [])
    query_template = openalex_templates.get(template_id, "")
    
    if not query_template:
        continue  # Skip if no matching template is found
    
    # Replace ORCID values
    orcid_values = [f'"{e["orcid"][0]}"' for e in entities if "orcid" in e]  # Ensure ORCID is properly quoted
    for orcid in orcid_values:
        query_template = query_template.replace("<ORCID>", orcid, 1)  # Replace sequentially if multiple authors
    
    # Replace YEAR if applicable
    if "YEAR(NOW()) - <YEAR>" in query_template:
        sparql_dblp = question["query"]["sparql"]
        year_value = ""  # Default empty
        
        # Extract the year from the DBLP query if present
        if "YEAR(NOW())-" in sparql_dblp:
            year_value = sparql_dblp.split("YEAR(NOW())-")[-1].split(")")[0].strip()
        
        query_template = query_template.replace("<YEAR>", year_value)
    
    # Store translated query
    translated_queries.append({
        "id": question["id"],
        "template_id": template_id,
        "sparql_OpenAlex": query_template
    })

# Save the translated queries to a JSON file
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(translated_queries, f, indent=4, ensure_ascii=False)

print(f"Translated queries saved to {output_file}")


Translated queries saved to translated_queries_DBLP_OpenAlex.json


Run Queries on OpenAlex and store the results which will be the gold label answers

In [ ]:
# File paths
input_file = "translated_queries_DBLP_OpenAlex.json"
output_file = "OpenAlex_100_results.json"
OpenAlex_endpoint = "https://semopenalex.org/sparql"

# Load translated queries
with open(input_file, "r", encoding="utf-8") as f:
    translated_queries = json.load(f)

# Initialize counters
success_count = 0
fail_count = 0
query_results = []
failed_queries = []

# Function to execute SPARQL query
def run_sparql_query(query):
    headers = {"Accept": "application/json"}
    params = {"query": query, "format": "json"}
    response = requests.get(OpenAlex_endpoint, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json(), True
    else:
        return response.text, False

# Execute each query
for query_data in translated_queries:
    query = query_data.get("sparql_OpenAlex", "")
    if not query:
        continue
    
    result, success = run_sparql_query(query)
    if success:
        success_count += 1
    else:
        fail_count += 1
        failed_queries.append({
            "id": query_data["id"],
            "template_id": query_data["template_id"],
            "sparql_query": query,
            "error": result
        })
    
    query_results.append({
        "id": query_data["id"],
        "template_id": query_data["template_id"],
        "sparql_query": query,
        "success": success,
        "result": result
    })

# Save results to JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(query_results, f, indent=4, ensure_ascii=False)

# Print statistics
print(f"Total queries executed: {len(translated_queries)}")
print(f"Successful queries: {success_count}")
print(f"Failed queries: {fail_count}")

# Print failed queries explicitly
if failed_queries:
    print("\nFailed Queries:")
    for failed in failed_queries:
        print(f"ID: {failed['id']}, Template: {failed['template_id']}")
        print(f"Query: {failed['sparql_query']}")
        print(f"Error: {failed['error']}\n")

Total queries executed: 100
Successful queries: 100
Failed queries: 0
